In [1]:
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import transformations
import config
%matplotlib inline

In [2]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery('2,7,8,9')", conn)


starter_data = RideWaits.copy()

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
RideWaits.head()

In [3]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop([column_name], axis = 1)
    return df

def get_shift(day, steps):
    previous_steps = {}
    for i in range(1,1+steps):
        current_steps = []
        test_day_current = day.reset_index()
        for index,row in test_day_current.iterrows():
            if index in list(range(i)):
                current_steps.append(0)
            else:
                current_steps.append(test_day_current.loc[index - i,'Wait'])
        
        name = "previous_step"+str(i)
        previous_steps[name] = current_steps
    
    for key,value in previous_steps.items():
        day[key] = value
        
    return day
    
    
def shift_data(ride_data, shift_range):
    new_data_frame = pd.DataFrame()
    distinct_rides = list(ride_data['RideId'].unique())
    for ride in distinct_rides:
        this_ride = ride_data[ride_data['RideId'] == ride]
        day_list = list(this_ride['Date'].unique())
        for day in day_list:
            day_data = this_ride[this_ride['Date'] == day]
            new_data = get_shift(day_data, shift_range)
            new_data_frame = pd.concat([new_data_frame, new_data])

    return new_data_frame

def model_transformation(data, num_shifts, start_day = True, by_ride = True):
    ride_waits = data
    
    if by_ride:
        important_columns = ['Wait','DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month','TimeSinceOpen','TimeSinceMidday','MinutesSinceOpen']
        dummy_columns = ['DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month']
    else:
        important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
        ride_waits = ride_waits[ride_waits['Location'] != ""]
        dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
        ride_waits = ride_waits.drop(['Name'], axis = 1)
    ride_waits = ride_waits[important_columns]
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    correlation = ride_waits.corr()['Wait']
    key_correlations = correlation[abs(correlation) > .005]
    important_cols = list(key_correlations.index)
    shift_columns = []
    if start_day == False:
        shift_columns = ["previous_step" + str(x+1)  for x in range(num_shifts)]
    important_cols = important_cols + ["Wait","MinutesSinceOpen"] + shift_columns
    important_cols = [x for x in important_cols if x != "Weekend_0"]
    important_cols = list(set(important_cols))
    ride_waits_key = ride_waits[important_cols]
    
    
    return ride_waits_key



def new_data_transform(data, num_shifts, important_cols, start_day = True, by_ride = True):
    ride_waits = data
    
    if by_ride:
        important_columns = ['Wait','DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month','TimeSinceOpen','TimeSinceMidday','MinutesSinceOpen']
        dummy_columns = ['DayOfWeek','Weekend','inEMH','EMHDay','MagicHourType','Month']
    else:
        important_columns = ['RideId','Date', 'Wait','Name','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','TimeSinceOpen','TimeSinceMidday','MagicHourType','MinutesSinceOpen','Month']
        ride_waits = ride_waits[ride_waits['Location'] != ""]
        dummy_columns = ['RideId','Tier','Location','IntellectualProp','ParkId','DayOfWeek','Weekend','CharacterExperience','inEMH','EMHDay','MagicHourType','Month']
        ride_waits = ride_waits.drop(['Name'], axis = 1)
    

    ride_waits = ride_waits[important_columns]
    
    ride_waits = ride_waits.dropna(how = "any")
    
    if start_day == False:
        ride_waits = shift_data(ride_waits,num_shifts)
    for column in dummy_columns:
        ride_waits = create_dummies(ride_waits, column)
        
    missing_cols = [x for x in important_cols if x not in ride_waits.columns]
    for col in missing_cols:
        ride_waits[col] = 0
        
    return ride_waits

In [4]:
from datetime import datetime
from pytz import timezone
tz = timezone('US/Eastern')
dtime = datetime.now(tz)
dtime = dtime.replace(hour = 7,minute = 0, second = 0, microsecond = 0)
date = dtime.date()
time = dtime.time().strftime("%H:%M")
from datetime import datetime
from dateutil.relativedelta import relativedelta

def date_range(start_date, end_date, increment, period):
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result

end_time = dtime.replace(hour = 23, minute = 45, second = 0, microsecond = 0)
time_list = date_range(dtime, end_time, 15, 'minutes')
time_list = [x.time().strftime("%H:%M") for x in time_list]

In [5]:
def get_conf_interval(clf, df):
    conf_high_list = []
    conf_low_list = []
    for index, row in df.iterrows():
        current_row = df.loc[[index]]
        all_predictions = [estimator.predict(current_row) for estimator in clf.estimators_]
        mean = np.mean(all_predictions)
        pred_std = np.std(all_predictions)
        conf_high = (mean + 2*pred_std)
        conf_low = (mean - 2*pred_std)
        conf_high_list.append(conf_high)
        conf_low_list.append(conf_low)
    
    df['confidence_high'] = conf_high_list
    df['confidence_low'] = conf_low_list
    return df

In [6]:
def make_daily_prediction(current_ride,ride, time_list, best_params, todays_predictions):
    ride_predictions = {}
    current_ride_fm = current_ride.copy()
    current_ride_fm = transformations.transformData(current_ride_fm)
    #print(current_ride.shape[0])
    #print(current_ride.columns)
    model_data = model_transformation(current_ride_fm, 1)
    important_columns = [x for x in model_data.columns if x != "Wait"]
    clf = RandomForestRegressor(**best_params)
    #scores = cross_val_score(clf, model_data[important_columns],model_data['Wait'], scoring = "neg_median_absolute_error", cv = 3)
    #ride_score = scores.mean()
    #ride_predictions['score'] = ride_score
    #print(model_data.head())
    clf.fit(model_data[important_columns], model_data['Wait'])
    predictions_frame = pd.DataFrame()
    ride_starter = current_ride.iloc[[0]]
    
    predictions_frame = pd.concat([ride_starter]*len(time_list),ignore_index = True)
    predictions_frame['Time'] = time_list
    predictions_frame = transformations.transformData(predictions_frame)
    # print(predictions_frame)
    #predictions_frame = transformations.transformData(predictions_frame)
    model_predictions_frame = new_data_transform(predictions_frame, 3, important_columns)
    predictions_frame['predicted_wait'] = clf.predict(model_predictions_frame[important_columns])
    model_predictions_frame = get_conf_interval(clf,model_predictions_frame[important_columns])
    predictions_frame['confidence_high'] = model_predictions_frame['confidence_high']
    predictions_frame['confidence_low'] = model_predictions_frame['confidence_low']
    
    
    
    
#     for time in time_list:
#         ride_starter['Time'] = time
#         row_data = ride_starter.copy()
#         #print(row_data)
#         try:
#             row_data = transformations.transformData(row_data)
#         except:
#             continue
#         #print(row_data)
#         predictions_frame = pd.concat([predictions_frame,row_data])
#         #print(predictions_frame.iloc[[predictions_frame.shape[0]-1]])
#         model_predictions_frame = new_data_transform(predictions_frame,3, important_columns)
#         predictions_frame['Wait'] = clf.predict(model_predictions_frame[important_columns])
#         #print(predictions_frame)
    
    ride_predictions['predictions'] = predictions_frame
    todays_predictions[ride] = ride_predictions

In [7]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
best_params = {'criterion': 'mse',
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

rides = list(set(starter_data['Name']))
#rides = ["Expedition Everest - Legend of the Forbidden Mountain","Gran Fiesta Tour Starring The Three Caballeros","Star Wars Launch Bay: Encounter Kylo Ren"]
global todays_predictions
todays_predictions = {}

import threading
num_threads = len(rides)

threads = []
for i in range(num_threads):
    print(rides[i-1])
    ride = rides[i-1]
    current_ride = starter_data.copy()
    current_ride = starter_data[current_ride['Name'] == ride]
    process = threading.Thread(target = make_daily_prediction, args = [current_ride,ride,time_list, best_params, todays_predictions])
    process.start()
    threads.append(process)

for process in threads:
    process.join()


Rock n Roller Coaster Starring Aerosmith
Muppet*Vision 3D
Kali River Rapids
Under the Sea ~ Journey of The Little Mermaid
Big Thunder Mountain Railroad
Spaceship Earth


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["Month"] = RideWaits["Date"].dt.month
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

The Magic Carpets of Aladdin


C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["DayOfWeek"] = pd.Categorical(RideWaits["DayOfWeek"])


The Many Adventures of Winnie the Pooh
Kilimanjaro Safaris
Prince Charming Regal Carrousel
Meet Daring Disney Pals as Circus Stars at Petes Silly Sideshow
Test Track
its a small world
BB-8 Astromech on Duty


...

The Hall of Presidents
Space Mountain


Exception in thread Thread-57:
Traceback (most recent call last):
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-f901591ffd33>", line 14, in make_daily_prediction
    clf.fit(model_data[important_columns], model_data['Wait'])
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py", line 315, in fit
    random_state=random_state)
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\base.py", line 125, in _make_estimator
    estimator = clone(self.base_estimator_)
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\base.py", line 63, in clone
    params_set = new_object.get_params(deep=False)
  File "C:\Users\

TriceraTop Spin


Exception in thread Thread-74:
Traceback (most recent call last):
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-f901591ffd33>", line 14, in make_daily_prediction
    clf.fit(model_data[important_columns], model_data['Wait'])
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\forest.py", line 315, in fit
    random_state=random_state)
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\ensemble\base.py", line 125, in _make_estimator
    estimator = clone(self.base_estimator_)
  File "C:\Users\chrisA\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\base.py", line 59, in clone
    new_object_params = estimator.get_params(deep=False)
  File "C:\

Wall time: 2min 25s


In [8]:
metric = []
for key, value in todays_predictions.items():
    print(str(key) + " : " + str(value['predictions'].shape[0]))
    

Star Wars Launch Bay Theater : 43
Tom Sawyer Island : 47
A Pirates Adventure ~ Treasures of the Seven Seas : 55
Walt Disneys Enchanted Tiki Room : 55
Walt Disneys Carousel of Progress : 55
Royal Sommerhus : 47
BB-8 Astromech on Duty : 43
Rock n Roller Coaster Starring Aerosmith : 43
Muppet*Vision 3D : 43
Celebrity Spotlight : 43
Kali River Rapids : 43
Star Wars Launch Bay: Encounter Kylo Ren : 43
Avatar Flight of Passage : 43
Kilimanjaro Safaris : 43
Alien Swirling Saucers : 43
Its Tough to be a Bug! : 43
Mickey and Minnie Starring in Red Carpet Dreams : 43
Meet Chewbacca at Star Wars Launch Bay : 43
Test Track : 47
Star Tours- The Adventures Continue : 43
Meet Favorite Disney Pals at Adventurers Outpost : 43
DINOSAUR : 43
Primeval Whirl : 43
Frozen Ever After : 47
Journey Into Imagination With Figment : 47
Living with the Land : 47
Gran Fiesta Tour Starring The Three Caballeros : 47
Expedition Everest - Legend of the Forbidden Mountain : 43
The Seas with Nemo and Friends : 47
The Twil

In [9]:
all_predictions = pd.DataFrame()
for key,value in todays_predictions.items():
    current_frame = value['predictions']
    current_frame = current_frame[['RideId','Time','predicted_wait','confidence_high','confidence_low']]
    current_frame.columns = ['RideId','Time','PredictedWait','ConfidenceHigh','ConfidenceLow']
    current_frame['PredictedWait'] = [int(str(x).split(".")[0]) for x in current_frame['PredictedWait']]
    current_frame['ConfidenceHigh'] = [int(str(x).split(".")[0]) for x  in current_frame['ConfidenceHigh']]
    current_frame['ConfidenceLow'] = [int(str(x).split(".")[0]) for x in current_frame['ConfidenceLow']]
    all_predictions = pd.concat([all_predictions, current_frame])

In [10]:
RideIds = list(set(RideWaits['RideId']))

In [11]:
rides_not_predicted = [x for x in RideIds if x not in list(set(all_predictions["RideId"]))]

In [12]:
rides_not_predicted

[17564219, 15743682, 209857, 18904138, 80010192, 18665185, 80069754]

In [16]:
if len(rides_not_predicted) > 0:
    best_params = {'criterion': 'mse',
     'max_depth': 10,
     'max_features': 'auto',
     'min_samples_leaf': 5,
     'min_samples_split': 2,
     'n_estimators': 100}

    #rides = list(set(starter_data['Name']))
    rides = rides_not_predicted
    global todays_predictions
    todays_predictions = {}

    import threading
    num_threads = len(rides)

    threads = []
    for i in range(num_threads):
        print(rides[i-1])
        ride = rides[i-1]
        current_ride = starter_data.copy()
        current_ride = starter_data[current_ride['RideId'] == ride]
        process = threading.Thread(target = make_daily_prediction, args = [current_ride,ride,time_list, best_params, todays_predictions])
        process.start()
        threads.append(process)

    for process in threads:
        process.join()
        
    more_predictions = pd.DataFrame()
    for key,value in todays_predictions.items():
        current_frame = value['predictions']
        current_frame = current_frame[['RideId','Time','predicted_wait','confidence_high','confidence_low']]
        current_frame.columns = ['RideId','Time','PredictedWait','ConfidenceHigh','ConfidenceLow']
        current_frame['PredictedWait'] = [int(str(x).split(".")[0]) for x in current_frame['PredictedWait']]
        current_frame['ConfidenceHigh'] = [int(str(x).split(".")[0]) for x  in current_frame['ConfidenceHigh']]
        current_frame['ConfidenceLow'] = [int(str(x).split(".")[0]) for x in current_frame['ConfidenceLow']]
        more_predictions = pd.concat([more_predictions, current_frame])
    
    
    all_predictions = pd.concat([all_predictions, more_predictions])

80010192
15743682


In [17]:
rides_not_predicted = [x for x in RideIds if x not in list(set(all_predictions["RideId"]))]
rides_not_predicted

[]

In [21]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)
cur = conn.cursor()
query = "delete from DisneyDB.Ride_Waits_Today_Predicted where RideId > 0"
cur.execute(query)
conn.commit()

for index,row in all_predictions.iterrows():
    query = "insert into DisneyDB.Ride_Waits_Today_Predicted (RideId, Time, PredictedWait, ConfidenceHigh, ConfidenceLow) values (%i, '%s', %i, %i, %i)" %(row['RideId'], row['Time'], row['PredictedWait'], row['ConfidenceHigh'],row['ConfidenceLow'])
    cur.execute(query)
    conn.commit()